<a href="https://colab.research.google.com/github/GuiSamuka/IC-CoRoT_Kepler/blob/main/00_Reading_and_Plotting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Manipulating and Viewing .fits files**

### **Bibliotecas**

In [1]:
# Para manipulação dos arquivos .fits
from astropy.io import fits

# Para manipulação dos dados obtidos pelos arquivos .fits
import numpy as np

# Para utilizar funções de DataFrame
import pandas as pd

# Para plotar o gráfico da Curva de Luz
import plotly.express as px

Primeiramente, precisamos importar o arquivo .fits

E logo podemos ver que ele é do tipo HDUList

In [22]:
image_file = fits.open('/content/drive/MyDrive/00_IC/dataset_exoplanets_confirmed/EN2_STAR_CHR_0101086161_20070516T060226_20071005T074409.fits')
print(type(image_file))

<class 'astropy.io.fits.hdu.hdulist.HDUList'>


Os arquivos .fits possuem diversas extensões, como podemos ver pelo *len(image_file)*, nosso arquivo possui 4 extensões. 
Nos interessa apenas a 3º extensão visto que ela apresenta apenas os dados do DATEBARRT e do WHITEFLUXSYS.


In [23]:
len(image_file)

4

Dessa forma, obtemos os dados contidos na extesão 3 do arquivo .fits e observamos que são do tipo FITS_rec.

In [28]:
scidata = image_file[3].data
print(type(scidata))

print("\n")
scidata

<class 'astropy.io.fits.fitsrec.FITS_rec'>




FITS_rec([(54236.75758185, 112626.77,   0),
          (54236.76350826, 112605.61,   0),
          (54236.76943468, 112771.5 ,   8), ...,
          (54378.80910033, 112496.13, 256),
          (54378.81502574, 112344.83,   0),
          (54378.82095114, 112318.5 ,  80)],
         dtype=(numpy.record, [('DATEBARTT', '>f8'), ('WHITEFLUXSYS', '>f4'), ('STATUSSYS', '>i4')]))

Para manipulação do *scidata*, opta-se por transforma-lo em array, utilizando a biblioteca *NumPy*

Nesse ponto, chegamos a um problema que vale a pena ser comentado.

O arquivo original, quando transformado para array, segundo o código:
```
# x = np.array(scidata)
```
apresenta o seguinte erro ao ser plotado futuramente, utilizando a biblioteca *Plotly*: 

**ValueError: Big-endian buffer not supported on little-endian compiler**

Esse erro ocorre quando os dados que estão sendo trabalhados foram criados em uma máquina com uma ordem de bits diferente da que está executando o *Python*. 
Para lidar com esse problema, devemos converter a matriz NumPy para a ordem de bits do sistema nativo antes de tranforma-la em um DataFrame ou uma Series.

Assim, utiliza-se os métodos *byteswap()* e *newbyteorder*().

Referência: https://pandas.pydata.org/pandas-docs/stable/user_guide/gotchas.html#byte-ordering-issues


In [29]:
x = np.array(scidata).byteswap().newbyteorder()

Com os dados normalizados, os transformamos em DataFrame e em seguida separamos a parte que nos interessa, ou seja, apenas as colunas DATABARRT e WHITEFLUXSYS.

In [30]:
data = pd.DataFrame(x)

In [31]:
data.head()

,DATEBARTT,WHITEFLUXSYS,STATUSSYS
0,54236.757582,112626.773438,0
1,54236.763508,112605.609375,0
2,54236.769435,112771.500000,8
3,54236.775361,113113.601562,0
4,54236.781288,112621.789062,256


In [32]:
data = data[['DATEBARTT', 'WHITEFLUXSYS']]

In [33]:
data.head()

,DATEBARTT,WHITEFLUXSYS
0,54236.757582,112626.773438
1,54236.763508,112605.609375
2,54236.769435,112771.500000
3,54236.775361,113113.601562
4,54236.781288,112621.789062


### **Plotando a Curva de Luz**

Plota-se a série temporal utilizando o *Plotly* como um gráfico de linha.

In [18]:
fig = px.line(data, x='DATEBARTT', y='WHITEFLUXSYS', title='Light Curve')
fig.show()

### **Saving data as .csv**

Por fim, salvamos os dados obtidos no .fits como csv, visando facilitar as futuras operações.

In [34]:
data.to_csv('data_exo.csv')